In [1]:
from data.dataset import DatasetLoader
dataset = DatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_transformed'
)
dataset[0].pressure.size(1)

201

In [3]:
# Test batching
from preprocessing.batching_v2 import get_batch_graphs

batched_dataset = []
for data in dataset:
    batched_dataset += get_batch_graphs(
        data=data,
        batch_size=1000,
        batch_n_times=50,
        recursive=True
    )

len(batched_dataset)

[TorchGraphData(x=[1048, 3], edge_index=[2, 1042], edge_attr=[1042, 7], pressure=[1048, 51], flowrate=[1042, 51], velocity=[1042, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1042], edge_attr=[1042, 7], pressure=[1048, 51], flowrate=[1042, 51], velocity=[1042, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1042], edge_attr=[1042, 7], pressure=[1048, 51], flowrate=[1042, 51], velocity=[1042, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1042], edge_attr=[1042, 7], pressure=[1048, 51], flowrate=[1042, 51], velocity=[1042, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1046], edge_attr=[1046, 7], pressure=[1048, 51], flowrate=[1046, 51], velocity=[1046, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1046], edge_attr=[1046, 7], pressure=[1048, 51], flowrate=[1046, 51], velocity=[1046, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1046], edge_attr=[1046, 7], pressure=[1048, 51], flowrate=[1046, 51], velocity=[1046, 51]),
 TorchGraphData(x=[1048, 3], edge_index=[2, 1046